# Beispielcode für ein NN das Ziffern zu unterscheiden lernt
Beachten Sie insbesondere die Zelle in der MNISTClassifier definiert wird.


In [7]:
import time
import random
import os
import struct
import argparse
import numpy as np
import dynet as dy
import ipywidgets as widgets
import PIL.Image
dy.sigmoid = dy.logistic

In [8]:
# minimally adapted from https://gist.github.com/akesling/5358964
def read_mnist(dataset, path="../mnist-dynet"):
    if dataset == "training":
        fname_img = os.path.join(path, "train-images-idx3-ubyte")
        fname_lbl = os.path.join(path, "train-labels-idx1-ubyte")
    elif dataset == "testing":
        fname_img = os.path.join(path, "t10k-images-idx3-ubyte")
        fname_lbl = os.path.join(path, "t10k-labels-idx1-ubyte")
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, "rb") as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        labels = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, "rb") as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        images = np.multiply(
            np.fromfile(fimg, dtype=np.uint8).reshape(len(labels), rows*cols),
            1.0 / 255.0)

    get_instance = lambda idx: (labels[idx], images[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(labels)):
        yield get_instance(i)
        
training = [(lbl, img) for (lbl, img) in read_mnist("training")]
testing = [(lbl, img) for (lbl, img) in read_mnist("testing")]

In [9]:
HIDDEN_DIM1 = 32
HIDDEN_DIM2 = 16 
DROPOUT_RATE = 0.5

In [10]:
class MNISTClassify(object):
    '''Classifier with two hidden layers'''
    def __init__(self, m):
        input_size = 28 * 28
        self.W1 = m.add_parameters((HIDDEN_DIM1, input_size))
        self.b1 = m.add_parameters((HIDDEN_DIM1, ))
        self.W2 = m.add_parameters((HIDDEN_DIM2, HIDDEN_DIM1))
        self.b2 = m.add_parameters((HIDDEN_DIM2, ))
        self.W3 = m.add_parameters((10, HIDDEN_DIM2))
        self.b3 = m.add_parameters((10, ))

    def __call__(self, x, isTrain=False):
        # erste innere Schicht:
        h1 = dy.sigmoid(self.W1 * x + self.b1)
        # zweite innere Schicht:
        h2 = dy.sigmoid(self.W2 * h1 + self.b2)
        # Ergebnisschicht:
        activation = dy.sigmoid(self.W3 * h2 + self.b3)
        return activation

In [13]:
m = dy.Model()
classify = MNISTClassify(m)
#sgd = dy.SimpleSGDTrainer(m, learning_rate=0.01)
sgd = dy.AdamTrainer(m)
dev_time = 0

start = time.time()
for epoch in range(1,5):
    random.shuffle(training)
    print(("Epoch {} starting".format(epoch)))
    # train model:
    for cls, img in training:
        dy.renew_cg()
        # Bild als Eingabe aufbereiten
        x = dy.inputVector(img)
        # Outputaktivierung berechnen
        output = classify(x, isTrain=True)
        # "idealen" Output (überall 0, nur gewählte Klasse ist 1)
        ideal = np.zeros(10)
        ideal[cls] = 1
        ideal = dy.inputVector(ideal)
        # Kostenfunktion berechnen
        loss = 0.5 * dy.sum_elems((output - ideal) ** 2)
        # Kosten den einzelnen Parametern zuordnen
        loss.backward()
        # Parameter entsprechend anpassen
        sgd.update()

    # test performance of classifier:
    confusion = [[0 for _ in range(10)] for _ in range(10)]
    correct = 0
    dev_start = time.time()
    for cls, img in testing:
        dy.renew_cg()
        x = dy.inputVector(img)
        output = classify(x, isTrain=False)
        prediction = max([(v,i) for (i,v) in enumerate(output.value())])[1]
        if cls == prediction:
            correct += 1
        confusion[prediction][cls] += 1
    dev_end = time.time()
    acc = float(correct) / len(testing)
    dev_time += dev_end - dev_start
    print(("Held out accuracy {} ({} instances/sec)".format(
        acc, len(testing) / (dev_end - dev_start))))
    print('   ' + ''.join(('T'+str(x)).ljust(6) for x in range(10)))
    for p, row in enumerate(confusion):
        s = 'P' + str(p) + ' '
        s += ''.join(str(col).ljust(6) for col in row)
        print(s)

    end = time.time()
    print(("instances per sec: {}".format(
                (epoch * len(training)) / (end - start - dev_time))))

Epoch 1 starting
Held out accuracy 0.9197 (3187.3766538922464 instances/sec)
   T0    T1    T2    T3    T4    T5    T6    T7    T8    T9    
P0 957   0     10    2     0     16    11    2     4     4     
P1 0     1091  2     0     1     1     3     5     2     3     
P2 3     5     939   24    7     11    14    28    8     0     
P3 2     8     27    916   0     22    2     12    26    14    
P4 0     1     11    0     915   8     7     10    7     22    
P5 6     1     5     47    2     795   32    2     49    26    
P6 6     3     7     0     14    8     876   0     6     3     
P7 2     1     9     10    2     5     1     927   2     8     
P8 4     24    20    9     8     21    12    4     869   17    
P9 0     1     2     2     33    5     0     38    1     912   
instances per sec: 1987.5762740493692
Epoch 2 starting
Held out accuracy 0.9285 (3434.3933081184928 instances/sec)
   T0    T1    T2    T3    T4    T5    T6    T7    T8    T9    
P0 945   0     6     0     0     10    7

In [8]:
class MNISTClassify(object):
    def __init__(self, m):
        input_size = 28 * 28
        self.W1 = m.add_parameters((HIDDEN_DIM1, input_size))
        self.h1bias = m.add_parameters((HIDDEN_DIM1, ))
        if HIDDEN_DIM2:
            self.W2 = m.add_parameters((HIDDEN_DIM2, HIDDEN_DIM1))
            self.h2bias = m.add_parameters((HIDDEN_DIM2 if HIDDEN_DIM2 else HIDDEN_DIM1, ))
        self.W3 = m.add_parameters((10, HIDDEN_DIM2 if HIDDEN_DIM2 else HIDDEN_DIM1))

    def __call__(self, x, isTrain=False):
        h1 = dy.sigmoid(self.W1 * x + self.h1bias)
        if isTrain:
            h1 = dy.dropout(h1, DROPOUT_RATE)
        if HIDDEN_DIM2:
            h2 = dy.sigmoid(self.W2 * h1 + self.h2bias)
            if isTrain:
                h2 = dy.dropout(h2, DROPOUT_RATE)
        activation = dy.sigmoid(self.W3 * (h2 if HIDDEN_DIM2 else h1))
        return activation

In [12]:
class MNISTClassify(object):
    def __init__(self, m):
        input_size = 28 * 28
        self.W1 = m.add_parameters((HIDDEN_DIM1, input_size))
        self.b1 = m.add_parameters((HIDDEN_DIM1, ))
        self.W2 = m.add_parameters((HIDDEN_DIM2, HIDDEN_DIM1 + input_size))
        self.b2 = m.add_parameters((HIDDEN_DIM2, ))
        self.W3 = m.add_parameters((10, HIDDEN_DIM2))
        self.b3 = m.add_parameters((10, ))

    def __call__(self, x, isTrain=False):
        # erste innere Schicht:
        h1 = dy.sigmoid(self.W1 * x + self.b1)
        # zweite innere Schicht:
        h2_input = dy.concatenate([x, h1])
        h2 = dy.sigmoid(self.W2 * h2_input + self.b2)
        if isTrain:
            h2 = dy.dropout(h2, DROPOUT_RATE)
        # Ergebnisschicht:
        activation = dy.sigmoid(self.W3 * h2 + self.b3)
        return activation

In [6]:
import io
buffer = io.BytesIO()
PIL.Image.fromarray((255*training[0][1]).reshape(28, 28)).convert('L').save(buffer, format="PNG")
widgets.Image(value=buffer.getvalue(),width=28,height=28)


Widget Javascript not detected.  It may not be installed or enabled properly.
